# EDEKA Analytics Application Prototype

This notebook demonstrates a prototype for analyzing product performance and customer behavior for EDEKA supermarket chain. It includes:

1. Product Sales Analysis - Track which products sell fastest and analyze sales by category
2. Customer Purchase Behavior - Identify repeat customers and analyze purchase patterns
3. Data Privacy Considerations - Demonstrate data anonymization for GDPR compliance
4. Visualization Techniques - Create charts for the dashboard

The analysis in this notebook will serve as the foundation for a full analytics application with a React frontend and Python/Flask backend.

## 1. Import Required Libraries

Import Python libraries for data analysis, visualization, and API prototyping:

In [1]:
# Import data processing libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import hashlib

# Import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Import Flask for API prototyping
from flask import Flask, jsonify

# Configure visualization settings
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(style="whitegrid")
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'matplotlib'

## 2. Load and Explore Sample Data

For this prototype, we'll generate sample data that follows the structure outlined in the requirements:

- **Categories**: Product categories like fruits, dairy, meat, etc.
- **Products**: Individual products with ID, name, category, and quality
- **Customers**: Customer profiles with card ID, name, nationality, and age
- **Bills**: Transaction records with bill ID, card ID, and timestamp
- **BillItems**: Individual items in each bill with product ID, quantity, and price

Let's generate and explore this sample data:

In [ ]:
# Create sample Categories data
categories_data = {
    'category_id': list(range(1, 11)),
    'name': [
        'Fruits & Vegetables',
        'Dairy',
        'Meat',
        'Bakery',
        'Frozen Foods',
        'Beverages',
        'Snacks',
        'Household',
        'Health & Beauty',
        'Deli'
    ]
}

categories_df = pd.DataFrame(categories_data)
print("Categories Dataset:")
display(categories_df)

In [ ]:
# Create sample Products data with quality rating (1-5)
np.random.seed(42)  # For reproducibility

# Sample products by category
product_names = {
    1: ['Apples', 'Bananas', 'Carrots', 'Tomatoes', 'Spinach', 'Potatoes', 'Onions', 'Broccoli', 'Strawberries', 'Grapes'],
    2: ['Milk', 'Cheese', 'Yogurt', 'Butter', 'Cream', 'Sour Cream', 'Cottage Cheese', 'Cream Cheese', 'Ice Cream', 'Whipped Cream'],
    3: ['Chicken', 'Beef', 'Pork', 'Turkey', 'Lamb', 'Ground Beef', 'Sausages', 'Bacon', 'Ham', 'Steak'],
    4: ['Bread', 'Rolls', 'Baguette', 'Croissants', 'Donuts', 'Muffins', 'Cake', 'Cookies', 'Pie', 'Pastries'],
    5: ['Frozen Pizza', 'Ice Cream', 'Frozen Vegetables', 'Frozen Meals', 'Frozen Fruit', 'Frozen Fish', 'Frozen Desserts', 'Frozen Breakfast', 'Frozen Snacks', 'Frozen Meat'],
    6: ['Water', 'Soda', 'Juice', 'Coffee', 'Tea', 'Beer', 'Wine', 'Energy Drinks', 'Sports Drinks', 'Milk Alternatives'],
    7: ['Chips', 'Crackers', 'Pretzels', 'Popcorn', 'Nuts', 'Chocolate', 'Candy', 'Granola Bars', 'Trail Mix', 'Cookies'],
    8: ['Cleaning Supplies', 'Paper Towels', 'Toilet Paper', 'Laundry Detergent', 'Dish Soap', 'Trash Bags', 'Air Freshener', 'Sponges', 'Bleach', 'All-Purpose Cleaner'],
    9: ['Shampoo', 'Conditioner', 'Soap', 'Toothpaste', 'Deodorant', 'Lotion', 'Sunscreen', 'Razors', 'Makeup', 'Vitamins'],
    10: ['Sliced Meat', 'Sliced Cheese', 'Salads', 'Olives', 'Hummus', 'Prepared Meals', 'Sandwiches', 'Sushi', 'Rotisserie Chicken', 'Soup']
}

# Generate product data
products_data = {
    'product_id': [],
    'name': [],
    'category_id': [],
    'quality': [],  # Quality rating 1-5 (5 being highest)
    'price': [],
    'stock_date': []  # When the product was stocked
}

product_id = 1
for category_id, names in product_names.items():
    for name in names:
        products_data['product_id'].append(product_id)
        products_data['name'].append(name)
        products_data['category_id'].append(category_id)
        products_data['quality'].append(np.random.randint(1, 6))  # Random quality 1-5
        products_data['price'].append(round(np.random.uniform(0.99, 29.99), 2))  # Random price
        
        # Random stock date within the last 30 days
        stock_date = datetime.now() - timedelta(days=np.random.randint(1, 31))
        products_data['stock_date'].append(stock_date)
        
        product_id += 1

products_df = pd.DataFrame(products_data)
print("Products Dataset (first 10 rows):")
display(products_df.head(10))

In [ ]:
# Create sample Customers data
np.random.seed(42)  # For reproducibility

# Sample first names, last names, and nationalities
first_names = ['Emma', 'Noah', 'Olivia', 'Liam', 'Sophia', 'Mason', 'Ava', 'Jacob', 'Isabella', 'William',
               'Mia', 'Ethan', 'Charlotte', 'James', 'Amelia', 'Alexander', 'Abigail', 'Michael', 'Emily', 'Benjamin',
               'Lukas', 'Hannah', 'Matthias', 'Anna', 'Felix', 'Sarah', 'Maximilian', 'Laura', 'Jonas', 'Julia']

last_names = ['Schmidt', 'Müller', 'Schneider', 'Fischer', 'Weber', 'Meyer', 'Wagner', 'Becker', 'Schulz', 'Hoffmann',
              'Smith', 'Johnson', 'Williams', 'Jones', 'Brown', 'Davis', 'Miller', 'Wilson', 'Moore', 'Taylor',
              'Anderson', 'Thomas', 'Jackson', 'White', 'Harris', 'Martin', 'Thompson', 'Garcia', 'Martinez', 'Robinson']

nationalities = ['German', 'French', 'Italian', 'Spanish', 'Polish', 'Dutch', 'British', 'Turkish', 'Russian', 'Austrian']

# Generate customer data
num_customers = 100
customers_data = {
    'card_id': list(range(1001, 1001 + num_customers)),
    'first_name': np.random.choice(first_names, num_customers),
    'last_name': np.random.choice(last_names, num_customers),
    'nationality': np.random.choice(nationalities, num_customers),
    'age': np.random.randint(18, 80, num_customers),
    'registration_date': [datetime.now() - timedelta(days=np.random.randint(1, 365*3)) for _ in range(num_customers)]  # Random registration within last 3 years
}

customers_df = pd.DataFrame(customers_data)
print("Customers Dataset (first 10 rows):")
display(customers_df.head(10))

In [ ]:
# Create sample Bills and BillItems data
np.random.seed(42)  # For reproducibility

# Generate transaction data over the last 30 days
num_bills = 500
bill_start_id = 10001

bills_data = {
    'bill_id': list(range(bill_start_id, bill_start_id + num_bills)),
    'card_id': np.random.choice(customers_df['card_id'], num_bills),
    'timestamp': [datetime.now() - timedelta(days=np.random.randint(1, 31), 
                                           hours=np.random.randint(8, 21), 
                                           minutes=np.random.randint(0, 60)) 
                for _ in range(num_bills)],
    'store_id': np.random.randint(1, 11, num_bills)  # 10 different stores
}

bills_df = pd.DataFrame(bills_data)
bills_df = bills_df.sort_values('timestamp')

# Generate bill items data
bill_items_data = {
    'bill_item_id': [],
    'bill_id': [],
    'product_id': [],
    'quantity': [],
    'unit_price': [],
    'total_price': []
}

bill_item_id = 1
for bill_id in bills_df['bill_id']:
    # Each bill has 1-15 items
    num_items = np.random.randint(1, 16)
    # Select random products
    products = products_df.sample(num_items)
    
    for _, product in products.iterrows():
        quantity = np.random.randint(1, 6)  # 1-5 quantity per product
        unit_price = product['price']
        total_price = round(quantity * unit_price, 2)
        
        bill_items_data['bill_item_id'].append(bill_item_id)
        bill_items_data['bill_id'].append(bill_id)
        bill_items_data['product_id'].append(product['product_id'])
        bill_items_data['quantity'].append(quantity)
        bill_items_data['unit_price'].append(unit_price)
        bill_items_data['total_price'].append(total_price)
        
        bill_item_id += 1

bill_items_df = pd.DataFrame(bill_items_data)

print("Bills Dataset (first 10 rows):")
display(bills_df.head(10))

print("\nBill Items Dataset (first 10 rows):")
display(bill_items_df.head(10))

In [ ]:
# Calculate sold dates based on bills data to analyze sales velocity
# We'll create a dataframe that tracks when products were stocked and when they sold out

# Get total quantity sold for each product
product_sales = bill_items_df.groupby('product_id')['quantity'].sum().reset_index()
product_sales.rename(columns={'quantity': 'total_quantity_sold'}, inplace=True)

# Merge with products data to get stock dates
sales_velocity_df = pd.merge(product_sales, products_df[['product_id', 'name', 'category_id', 'stock_date']], on='product_id')

# Add category names
sales_velocity_df = pd.merge(sales_velocity_df, categories_df, on='category_id')

# Calculate a realistic sold_out_date based on sales volume and stock date
# Higher sales means product sells out faster
sales_velocity_df['initial_stock_quantity'] = np.random.randint(50, 200, len(sales_velocity_df))
sales_velocity_df['remaining_stock'] = sales_velocity_df['initial_stock_quantity'] - sales_velocity_df['total_quantity_sold']

# Estimate days to sell out based on quantity sold and initial stock
sales_velocity_df['days_to_sell_out'] = np.where(
    sales_velocity_df['remaining_stock'] <= 0,
    # If sold out, calculate realistic days based on total quantity sold
    np.ceil(sales_velocity_df['initial_stock_quantity'] / (sales_velocity_df['total_quantity_sold'] / 30 + 0.1)),
    # If not sold out, use None to indicate not sold out yet
    None
)

# Calculate sold out date for products that have sold out
sales_velocity_df['sold_out_date'] = [
    stock_date + timedelta(days=days) if days is not None else None
    for stock_date, days in zip(sales_velocity_df['stock_date'], sales_velocity_df['days_to_sell_out'])
]

# Flag products that have sold out
sales_velocity_df['is_sold_out'] = sales_velocity_df['sold_out_date'].notnull()

print("Sales Velocity Dataset (first 10 rows):")
display(sales_velocity_df.head(10))

## 3. Product Sales Analysis

Now that we have our sample data, let's analyze product sales to address Use Case 1:

1. **Sales Velocity**: Which products sell the fastest?
2. **Category Performance**: Which categories of products sell the fastest?
3. **Top Sellers**: Which products sell in the highest quantities?

These insights will help business owners make better inventory and stocking decisions.

In [ ]:
# 3.1 Analyze Sales Velocity - Which products sell the fastest?

# Filter to only include products that have sold out
sold_out_products = sales_velocity_df[sales_velocity_df['is_sold_out']].copy()

# Calculate sales velocity (days to sell out)
sold_out_products = sold_out_products.sort_values('days_to_sell_out')

print("Products by Sales Velocity (Fastest Selling First):")
display(sold_out_products[['product_id', 'name', 'name_y', 'initial_stock_quantity', 
                          'total_quantity_sold', 'days_to_sell_out']].head(10).rename(
                              columns={'name_y': 'category'}))

In [ ]:
# 3.2 Analyze Category Performance - Which categories sell the fastest?

# Calculate average days to sell out by category
category_velocity = sold_out_products.groupby('name_y')['days_to_sell_out'].agg(['mean', 'count']).reset_index()
category_velocity = category_velocity.sort_values('mean')
category_velocity.rename(columns={'name_y': 'category', 'mean': 'avg_days_to_sell_out', 'count': 'products_sold_out'}, inplace=True)

print("Categories by Sales Velocity (Fastest Selling First):")
display(category_velocity)

# Calculate total sales by category
category_sales = bill_items_df.merge(products_df[['product_id', 'category_id']], on='product_id')
category_sales = category_sales.merge(categories_df, on='category_id')
category_sales = category_sales.groupby('name')['quantity'].sum().reset_index()
category_sales = category_sales.sort_values('quantity', ascending=False)
category_sales.rename(columns={'name': 'category', 'quantity': 'total_quantity_sold'}, inplace=True)

print("\nCategories by Total Sales Volume:")
display(category_sales)

In [ ]:
# 3.3 Analyze Top Sellers - Which products sell in the highest quantities?

# Aggregate sales by product
product_quantity_sold = bill_items_df.groupby('product_id')['quantity'].sum().reset_index()
top_products = product_quantity_sold.merge(products_df[['product_id', 'name', 'category_id']], on='product_id')
top_products = top_products.merge(categories_df, on='category_id')
top_products = top_products.sort_values('quantity', ascending=False)

print("Top 15 Products by Sales Volume:")
display(top_products[['product_id', 'name_x', 'name_y', 'quantity']].head(15).rename(
    columns={'name_x': 'product', 'name_y': 'category', 'quantity': 'total_quantity_sold'}))

## 4. Customer Purchase Behavior Analysis

For Use Case 2, we need to analyze customer purchase behavior:

1. **Repeat Customers**: Which customers repeatedly buy the same products?
2. **Purchase Patterns**: Are there any patterns in purchase behavior by nationality or age?
3. **Customer Segmentation**: Identify different customer segments based on purchase behavior.

These insights will help with targeted marketing and personalized promotions.

In [ ]:
# 4.1 Analyze Repeat Customers - Which customers repeatedly buy the same products?

# Join bill items with bills to get customer information
customer_purchases = bill_items_df.merge(bills_df[['bill_id', 'card_id', 'timestamp']], on='bill_id')

# Count how many times each customer bought each product
customer_product_frequency = customer_purchases.groupby(['card_id', 'product_id'])['bill_id'].count().reset_index()
customer_product_frequency.rename(columns={'bill_id': 'purchase_count'}, inplace=True)

# Filter to only include repeat purchases (bought same product 2+ times)
repeat_purchases = customer_product_frequency[customer_product_frequency['purchase_count'] > 1]
repeat_purchases = repeat_purchases.sort_values('purchase_count', ascending=False)

# Add product and customer names
repeat_purchases = repeat_purchases.merge(products_df[['product_id', 'name']], on='product_id')
repeat_purchases = repeat_purchases.merge(customers_df[['card_id', 'first_name', 'last_name', 'nationality']], on='card_id')

print("Top Repeat Customers by Product:")
display(repeat_purchases.head(15).rename(columns={'name': 'product'}))

In [ ]:
# 4.2 Analyze Purchase Patterns by Nationality

# Join all the necessary tables
nationality_purchases = bill_items_df.merge(bills_df[['bill_id', 'card_id']], on='bill_id')
nationality_purchases = nationality_purchases.merge(customers_df[['card_id', 'nationality']], on='card_id')
nationality_purchases = nationality_purchases.merge(products_df[['product_id', 'category_id']], on='product_id')
nationality_purchases = nationality_purchases.merge(categories_df, on='category_id')

# Analyze category preferences by nationality
nationality_category_pref = nationality_purchases.groupby(['nationality', 'name'])['quantity'].sum().reset_index()
nationality_category_pref = nationality_category_pref.sort_values(['nationality', 'quantity'], ascending=[True, False])
nationality_category_pref.rename(columns={'name': 'category', 'quantity': 'total_quantity'}, inplace=True)

# Get top category for each nationality
top_categories_by_nationality = nationality_category_pref.loc[nationality_category_pref.groupby('nationality')['total_quantity'].idxmax()]

print("Top Product Category by Nationality:")
display(top_categories_by_nationality)

# Calculate relative preference (percentage of each category within nationality)
nationality_totals = nationality_category_pref.groupby('nationality')['total_quantity'].sum().reset_index()
nationality_category_pref = nationality_category_pref.merge(nationality_totals, on='nationality', suffixes=('', '_total'))
nationality_category_pref['percentage'] = (nationality_category_pref['total_quantity'] / nationality_category_pref['total_quantity_total'] * 100).round(2)

print("\nTop 3 Categories for Each Nationality:")
for nationality in nationality_category_pref['nationality'].unique():
    print(f"\n{nationality}:")
    display(nationality_category_pref[nationality_category_pref['nationality'] == nationality].sort_values('total_quantity', ascending=False).head(3))

In [ ]:
# 4.3 Customer Segmentation - Identify different customer segments

# Calculate total spending by customer
customer_spending = bill_items_df.merge(bills_df[['bill_id', 'card_id']], on='bill_id')
customer_spending = customer_spending.groupby('card_id')['total_price'].sum().reset_index()
customer_spending.rename(columns={'total_price': 'total_spending'}, inplace=True)

# Calculate purchase frequency (number of unique bills)
purchase_frequency = bills_df.groupby('card_id')['bill_id'].nunique().reset_index()
purchase_frequency.rename(columns={'bill_id': 'visit_count'}, inplace=True)

# Calculate average basket size
basket_size = bills_df.merge(bill_items_df, on='bill_id')
basket_size = basket_size.groupby(['card_id', 'bill_id'])['quantity'].sum().reset_index()
basket_size = basket_size.groupby('card_id')['quantity'].mean().reset_index()
basket_size.rename(columns={'quantity': 'avg_basket_size'}, inplace=True)

# Combine metrics for segmentation
customer_segments = customer_spending.merge(purchase_frequency, on='card_id')
customer_segments = customer_segments.merge(basket_size, on='card_id')
customer_segments = customer_segments.merge(customers_df[['card_id', 'first_name', 'last_name', 'nationality', 'age']], on='card_id')

# Calculate recency (days since last purchase)
last_purchase_date = bills_df.groupby('card_id')['timestamp'].max().reset_index()
last_purchase_date['days_since_last_purchase'] = (datetime.now() - last_purchase_date['timestamp']).dt.days
customer_segments = customer_segments.merge(last_purchase_date[['card_id', 'days_since_last_purchase']], on='card_id')

# Create simple segments based on spending and frequency
def create_segment(row):
    if row['total_spending'] > 500 and row['visit_count'] > 5:
        return 'High-Value Regular'
    elif row['total_spending'] > 500:
        return 'Big Spender'
    elif row['visit_count'] > 5:
        return 'Frequent Visitor'
    elif row['days_since_last_purchase'] < 7:
        return 'Recent Customer'
    else:
        return 'Occasional Shopper'

customer_segments['segment'] = customer_segments.apply(create_segment, axis=1)

# Show segment distribution
segment_counts = customer_segments.groupby('segment')['card_id'].count().reset_index()
segment_counts.rename(columns={'card_id': 'customer_count'}, inplace=True)
segment_counts['percentage'] = (segment_counts['customer_count'] / segment_counts['customer_count'].sum() * 100).round(2)

print("Customer Segments Distribution:")
display(segment_counts)

# Show examples of customers in each segment
print("\nCustomer Segment Examples:")
for segment in customer_segments['segment'].unique():
    print(f"\n{segment} Examples:")
    display(customer_segments[customer_segments['segment'] == segment].sort_values('total_spending', ascending=False).head(3)[
        ['card_id', 'first_name', 'last_name', 'total_spending', 'visit_count', 'avg_basket_size', 'days_since_last_purchase']])

## 5. Data Privacy: Anonymization

Since EDEKA is based in Germany, GDPR compliance is essential. Let's demonstrate how to anonymize customer data while still maintaining the ability to perform analytics:

1. **Hash Customer IDs**: Replace card_id with a hashed version that can't be reversed
2. **Remove Personal Information**: Remove first and last names from analytics data
3. **Aggregate Sensitive Data**: Present nationality data only in aggregated form

In [ ]:
# 5.1 Create anonymized version of customer data

def hash_id(id_value, salt="EDEKA_ANALYTICS"):
    """Create a hashed version of an ID that can't be reversed"""
    to_hash = f"{salt}_{id_value}"
    return hashlib.sha256(to_hash.encode()).hexdigest()[:16]  # Take first 16 chars for readability

# Create anonymized customer data
anonymized_customers = customers_df.copy()
anonymized_customers['hashed_card_id'] = anonymized_customers['card_id'].apply(hash_id)
anonymized_customers.drop(['first_name', 'last_name', 'card_id'], axis=1, inplace=True)

print("Anonymized Customer Data (GDPR Compliant):")
display(anonymized_customers.head(10))

# Create anonymized bills data
anonymized_bills = bills_df.copy()
anonymized_bills['hashed_card_id'] = anonymized_bills['card_id'].apply(hash_id)
anonymized_bills.drop(['card_id'], axis=1, inplace=True)

print("\nAnonymized Bills Data:")
display(anonymized_bills.head(10))

In [ ]:
# 5.2 Demonstrate aggregated nationality data for analytics

# This shows how to present nationality data in an aggregated form
# rather than linking it directly to individual customers

# Aggregated sales by nationality and product category
nationality_category_agg = nationality_purchases.groupby(['nationality', 'name'])['quantity'].sum().reset_index()
nationality_category_agg.rename(columns={'name': 'category', 'quantity': 'total_quantity'}, inplace=True)
nationality_category_agg = nationality_category_agg.sort_values(['nationality', 'total_quantity'], ascending=[True, False])

# Only show categories with significant sales (privacy protection)
min_threshold = 10  # Only show categories with at least 10 items sold
nationality_category_agg = nationality_category_agg[nationality_category_agg['total_quantity'] >= min_threshold]

print("Aggregated Sales by Nationality and Category (GDPR Compliant):")
display(nationality_category_agg.head(15))

# Age groups instead of specific ages
anonymized_customers['age_group'] = pd.cut(
    anonymized_customers['age'], 
    bins=[0, 25, 35, 50, 65, 100],
    labels=['18-25', '26-35', '36-50', '51-65', '65+']
)

age_group_counts = anonymized_customers.groupby(['nationality', 'age_group']).size().reset_index()
age_group_counts.rename(columns={0: 'customer_count'}, inplace=True)

# Only show groups with at least 3 customers (k-anonymity with k=3)
age_group_counts = age_group_counts[age_group_counts['customer_count'] >= 3]

print("\nAggregated Customer Demographics (GDPR Compliant):")
display(age_group_counts)

## 6. Visualizing Insights with Charts

For Use Case 3, we need to create visualizations that would be included in a dashboard. Let's create a variety of charts to visualize the insights we've found:

1. **Bar Chart**: Top-selling products
2. **Pie Chart**: Sales by product category
3. **Line Chart**: Customer purchase trends over time
4. **Grouped Bar Chart**: Sales by nationality
5. **Heatmap**: Product quality vs demand

In [ ]:
# 6.1 Bar Chart: Top-selling products

# Get top 10 products by sales volume
top10_products = top_products.head(10).rename(columns={'name_x': 'product', 'name_y': 'category'})

# Create a horizontal bar chart with Plotly
fig = px.bar(
    top10_products,
    y='product',
    x='quantity',
    color='category',
    orientation='h',
    title='Top 10 Products by Sales Volume',
    labels={'quantity': 'Units Sold', 'product': 'Product', 'category': 'Category'},
    height=500
)

fig.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    xaxis_title='Units Sold',
    yaxis_title='Product',
    legend_title='Category'
)

fig.show()

In [ ]:
# 6.2 Pie Chart: Sales by product category

# Create a pie chart of sales by category
fig = px.pie(
    category_sales,
    values='total_quantity_sold',
    names='category',
    title='Sales Distribution by Product Category',
    color_discrete_sequence=px.colors.qualitative.Pastel,
    hole=0.3
)

fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(
    legend_title='Category',
    legend=dict(orientation='h', yanchor='bottom', y=-0.1, xanchor='center', x=0.5)
)

fig.show()

In [ ]:
# 6.3 Line Chart: Customer purchase trends over time

# Aggregate sales by date
daily_sales = bills_df.merge(bill_items_df, on='bill_id')
daily_sales['date'] = daily_sales['timestamp'].dt.date
daily_sales = daily_sales.groupby('date')['quantity'].sum().reset_index()
daily_sales = daily_sales.sort_values('date')

# Create a line chart of daily sales volume
fig = px.line(
    daily_sales,
    x='date',
    y='quantity',
    title='Daily Sales Volume Trend',
    labels={'quantity': 'Units Sold', 'date': 'Date'},
    markers=True
)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Units Sold'
)

fig.show()

# Aggregate sales by category and date
category_daily_sales = bills_df.merge(bill_items_df, on='bill_id')
category_daily_sales = category_daily_sales.merge(products_df[['product_id', 'category_id']], on='product_id')
category_daily_sales = category_daily_sales.merge(categories_df, on='category_id')
category_daily_sales['date'] = category_daily_sales['timestamp'].dt.date
category_daily_sales = category_daily_sales.groupby(['date', 'name'])['quantity'].sum().reset_index()
category_daily_sales = category_daily_sales.rename(columns={'name': 'category'})

# Filter to top 5 categories for clarity
top_categories = category_sales.head(5)['category'].tolist()
category_daily_sales = category_daily_sales[category_daily_sales['category'].isin(top_categories)]

# Create a multi-line chart for top 5 categories
fig = px.line(
    category_daily_sales,
    x='date',
    y='quantity',
    color='category',
    title='Daily Sales Trend by Top 5 Categories',
    labels={'quantity': 'Units Sold', 'date': 'Date', 'category': 'Category'}
)

fig.update_layout(
    xaxis_title='Date',
    yaxis_title='Units Sold',
    legend_title='Category'
)

fig.show()